In [98]:
import os
import numpy as np
import pandas as pd
brand_domains = pd.read_csv('brand_domains.csv')
brand_domains['status'] = np.nan

for filename in os.listdir('./sem_source/'):
    brand = filename.split('.csv')[0].split('+')[1]
#     print(brand, 'bad' in filename)
    brand_domains.loc[brand_domains['Brand'] == brand ,'status'] = 'bad' if 'bad' in filename else 'good'


In [99]:
print(brand_domains['status'].value_counts(dropna=False))
brand_domains

NaN     19613
bad     16557
good     3607
Name: status, dtype: int64


,Brand,Domain,Category,status
0,KEVITA,www.kevita.com,REFRIGERATED JUICES & FUNCTIONAL BEVERAGES,good
1,GATORADE PRIME 01,www.gatorade.com,WELLNESS BARS & GELS,good
2,RICCOS,NaN,SHELF STABLE JUICES,NaN
3,KEMPS,it.qwe.wiki,REFRIGERATED CREAMS & CREAMERS,bad
4,BOSTON MARKET,www.bostonmarket.com,FROZEN ENTREES,good
...,...,...,...,...
39772,LOUISIANA SELECT FOODS,www.louisianaselectfoods.com,FROZEN ENTREES,NaN
39773,DE YOUNGS,NaN,SHELF STABLE SEASONINGS,NaN
39774,WORLDS LARGEST LOLLIPOP,www.candy.com,SHELF STABLE CANDY,NaN
39775,HOT BREAD KITCHEN,NaN,BREAD & BAKED GOODS,NaN


###### Download Data for Brands With No Data

In [102]:
### DOWNLOAD DATA FOR MISSING BRANDS
missing_brands = brand_domains[brand_domains['status'].isna()]
print(missing_brands.shape)
missing_brands[missing_brands['Domain'].notna()]

(19613, 4)


,Brand,Domain,Category,status
10566,MIKE + IKE,www.mikeandike.com,SHELF STABLE CANDY,NaN
12069,THE SAFE + FAIR FOOD COMPANY,www.safeandfair.com,SHELF STABLE COLD CEREALS,NaN
19372,AQUA 9+,aquanineplus.com,SHELF STABLE WATER,NaN
23231,EVERHEMP+,everhemp.com,WELLNESS BARS & GELS,NaN
27803,SLAMMERS PROTEIN+,slammerssnacks.com,SHELF STABLE FRUITS & VEGETABLES,NaN
...,...,...,...,...
39770,HOPPY BUNNY,https://rmpalmer.com/,SHELF STABLE CANDY,NaN
39771,WHISPS,whisps.com,SHELF STABLE CHIPS & PRETZELS & SNACKS,NaN
39772,LOUISIANA SELECT FOODS,www.louisianaselectfoods.com,FROZEN ENTREES,NaN
39774,WORLDS LARGEST LOLLIPOP,www.candy.com,SHELF STABLE CANDY,NaN


In [106]:
# Create data dir
os.mkdir('raw_sem_missing_brands')

# Get Brands with domains from brands with no data
fetch_data_brands = missing_brands[missing_brands['Domain'].notna()].set_index('Brand')
fetch_data_brands

,Domain,Category,status
Brand,,,
MIKE + IKE,www.mikeandike.com,SHELF STABLE CANDY,NaN
THE SAFE + FAIR FOOD COMPANY,www.safeandfair.com,SHELF STABLE COLD CEREALS,NaN
AQUA 9+,aquanineplus.com,SHELF STABLE WATER,NaN
EVERHEMP+,everhemp.com,WELLNESS BARS & GELS,NaN
SLAMMERS PROTEIN+,slammerssnacks.com,SHELF STABLE FRUITS & VEGETABLES,NaN
...,...,...,...
HOPPY BUNNY,https://rmpalmer.com/,SHELF STABLE CANDY,NaN
WHISPS,whisps.com,SHELF STABLE CHIPS & PRETZELS & SNACKS,NaN
LOUISIANA SELECT FOODS,www.louisianaselectfoods.com,FROZEN ENTREES,NaN


In [134]:
# Lose https prefix from domains
fetch_data_brands.loc[:, 'Domain'] = fetch_data_brands['Domain'].str.strip('https://')

In [156]:
SEM_API_KEY = "dee26b8c660d5b17168acf360fb086c0"
base_url = "https://api.semrush.com/?"
type_ = "domain_rank_history"
database = 'us'

import requests
r = requests.get(url)

export_colums = "Ot,Lc,P0,Ds,Dn,Cp,Bm,Am,At,Ad,Rk,Or,Oc,Ad,At,Ac,Dt"

for brand in fetch_data_brands.index.values:
    
    print(brand, 'fetching data for domain: ', fetch_data_brands.loc[brand, 'Domain'], end='\n\n')
    
    domain = fetch_data_brands.loc[brand, 'Domain']
    url = f"{base_url}type={type_}&key={SEM_API_KEY}&export_columns={export_colums}&domain={domain}&database={database}"
    r = requests.get(url)
    
    if '/' in brand:
        brand = brand.replace('/', '|')
    
    text_file = open(f'./raw_sem_missing_brands/{brand}.csv', "w")

    text_file.write(r.text)

    text_file.close()
    # print(r.content)
    
    

MIKE + IKE fetching data for domain:  www.mikeandike.com

THE SAFE + FAIR FOOD COMPANY fetching data for domain:  www.safeandfair.com

AQUA 9+ fetching data for domain:  aquanineplus.com

EVERHEMP+ fetching data for domain:  everhemp.com

SLAMMERS PROTEIN+ fetching data for domain:  lammerssnacks.com

K+ fetching data for domain:  www.4plusnutrition.com

BARREL+BRINE fetching data for domain:  barrelnbrine.com

QKR HONEY/NUT OATS fetching data for domain:  www.quakeroats.com

CAKERY CLASSIC fetching data for domain:  cakeryclassic.com

PASTA MANCINI fetching data for domain:  www.pastamancini.com

21ST CENTURY BEAN & COMPANY fetching data for domain:  www.21stcenturybean.com

DESERT FARMS fetching data for domain:  desertfarms.com

KAIJA MATIJE fetching data for domain:  winelibrary.com

ROSA MEXICANO KITCHEN fetching data for domain:  www.rosamexicano.com

KILDARE PRINCESS fetching data for domain:  kildareprincess.ca

RISE UP COFFEE ROASTERS fetching data for domain:  riseupcoffee.co

MITY FRESH fetching data for domain:  www.ajcfood.com

BOMBOLO BAKERY fetching data for domain:  www.bombolobiscotti.com

VINCENZOS fetching data for domain:  www.vincenzosonline.com

KUL CHOCOLATE fetching data for domain:  www.kulchocolate.com

NUT MARKET fetching data for domain:  enutmarket.com.au

VALDIGRANO fetching data for domain:  www.valdigrano.com

ANNIES GINGER ELIXIR fetching data for domain:  www.anniesgingerelixir.com

TEMECULAR VALLEY HONEY COMPANY fetching data for domain:  emeculavalleyhoneycompany.com

BRIDGE CITY COFFEE fetching data for domain:  www.bridgecity.coffee

OTEAS fetching data for domain:  www.oteas.com

CAROLINA CHOCOLATE COMPANY fetching data for domain:  www.carolinachocolatecompany.com

AUNT LAURIES SWEETS fetching data for domain:  auntlauriessweets.com

EL GALEON fetching data for domain:  www.elgaleon.org

HUMBLE TEA CO fetching data for domain:  www.humbleteaco.com

PANDORO fetching data for domain:  ricette.giallozafferano.i

DON CHEMAS fetching

SNACKS SQUARED fetching data for domain:  walkers-snacks.co.uk

TROPICAL NUT & FRUIT fetching data for domain:  www.tropicalfoods.com

FRASER VALLEY GOURMET fetching data for domain:  www.fraservalleygourmet.com

PAIN DOR fetching data for domain:  www.paindor.fr

MY DADDYS CHEESECAKE fetching data for domain:  www.mydaddyscheesecake.com

REGENCY COFFEE fetching data for domain:  www.regencycoffee.co.uk

MAYWOOD FARMS fetching data for domain:  www.maywoodfarms.com

FINNOTTES fetching data for domain:  finnottes.com

MAHMOOD TEA fetching data for domain:  www.mahmoodtea.com

KUSARI KIDS fetching data for domain:  www.kusarinoodles.com

DALSTONS fetching data for domain:  dalstons.com

SAFE HARVEST fetching data for domain:  www.safeharvest.co.in

FRUITONS fetching data for domain:  www.trainafoods.com

KRAFT HEINZ YORK fetching data for domain:  www.kraftheinzcompany.com

INSTANT POT fetching data for domain:  instantpot.com

GINGERX fetching data for domain:  gingerxshots.com

SMALL A

LEVEL GROUND fetching data for domain:  levelground.com

WOLFFER fetching data for domain:  www.wolffer.com

KONA BAR fetching data for domain:  konabar.com

RUSTLERS fetching data for domain:  rustlersonline.com

BIOSTEEL fetching data for domain:  biosteel.com

BEACH DONUTS fetching data for domain:  beachdonutshop.com

HIBISCUS FOODS fetching data for domain:  www.hibiscusfoods.ca

ROCKS ULTIMATE MARINADE fetching data for domain:  madeinamericastore.com

EVERGREENS fetching data for domain:  evergreens.com

ITSUGAR SUGARPOVA fetching data for domain:  ugarpova.com

LEROUX CREEK fetching data for domain:  www.lerouxcreek.com

SPAGNOLOS fetching data for domain:  agsinfo.com

HANNAFORD BROS fetching data for domain:  www.hannaford.com

THE HOTEL KITCHEN fetching data for domain:  otelkitchen.org

BELLINGHAM BAY COFFEE ROASTERS fetching data for domain:  bellinghamcoffee.com

BUBBAS SALSA fetching data for domain:  www.bubbassalsa.com

GLYKA SWEETS fetching data for domain:  www.glyka

THE HAMPTONS HONEY COMPANY fetching data for domain:  www.hamptonshoney.com

DAN CAKE fetching data for domain:  dancake.

JJ fetching data for domain:  jj-foods.nl

BONGIORNO fetching data for domain:  op.ibuongiorno.com

GILHOOLEYS fetching data for domain:  www.gilhooleys.com

NUTRI N TASTY fetching data for domain:  www.nutrintasty.com

CEDAR PASTRIES fetching data for domain:  cedarfb.com

SWOON fetching data for domain:  www.tasteswoon.com

FEEL GOOD FOODS fetching data for domain:  feel-good-foods.com

PACIFIC PASTURES fetching data for domain:  www.pacificpastures.com

HERKYS fetching data for domain:  www.herkys.com

STEVITA fetching data for domain:  evitanaturals.com

HEALTHY BITES fetching data for domain:  www.healthybitesla.com

BUCKS COUNTY COFFEE ORGANIC fetching data for domain:  buckscountycoffee.com

UNCLE GENES fetching data for domain:  unclegenesoulandseafood.com

CULINARY KITCHENS fetching data for domain:  www.caesarskitchen.com

J SANTOS BAKERY fetching data fo

##### Check Data Format of Existing Files

In [159]:
sem_source_results_good_example = pd.read_csv('./sem_source/sem_source_results+KEMPS.csv')

In [160]:
sem_source_results_good_example

,domain,source_domain,display_date,country,device_type,traffic_share,traffic,traffic_channel,traffic_type,brand
0,it.qwe.wiki,NaN,2020-07-01,global,all,NaN,NaN,NaN,NaN,KEMPS
1,it.qwe.wiki,NaN,2020-07-01,global,all,NaN,NaN,NaN,NaN,KEMPS
2,it.qwe.wiki,NaN,2019-11-01,global,all,NaN,NaN,NaN,NaN,KEMPS
3,it.qwe.wiki,NaN,2019-11-01,global,all,NaN,NaN,NaN,NaN,KEMPS
4,it.qwe.wiki,NaN,2019-11-01,global,all,NaN,NaN,NaN,NaN,KEMPS
5,it.qwe.wiki,NaN,2019-11-01,global,all,NaN,NaN,NaN,NaN,KEMPS
6,it.qwe.wiki,NaN,2019-11-01,global,all,NaN,NaN,NaN,NaN,KEMPS
7,it.qwe.wiki,NaN,2019-11-01,global,all,NaN,NaN,NaN,NaN,KEMPS
8,it.qwe.wiki,NaN,2019-11-01,global,all,NaN,NaN,NaN,NaN,KEMPS
9,it.qwe.wiki,NaN,2019-11-01,global,all,NaN,NaN,NaN,NaN,KEMPS
